In [1]:
from pypbc import *
import hashlib
import time

In [2]:
stored_params = """type a
q 8780710799663312522437781984754049815806883199414208211028653399266475630880222957078625179422662221423155858769582317459277713367317481324925129998224791
h 12016012264891146079388821366740534204802954401251311822919615131047207289359704531102844802183906537786776
r 730750818665451621361119245571504901405976559617
exp2 159
exp1 107
sign1 1
sign0 1
"""

params = Parameters(param_string=stored_params)
pairing = Pairing(params)

In [3]:
def setup():
    g = Element.random(pairing, G1)
    h = Element.random(pairing, G1)
    zeta = Element.random(pairing, G2)
    
    return g, h, zeta

g, h, zeta = setup()
zeta

020FB02DFBF635752F80D1FBF546F9BF697298777DB9110DF9F33357DBCB23BCB492E19DB97EE19DFD00691F5293A1F9BBBC9BE8E8BDCB50621B9A7A41A4B36E99

In [4]:
def init(num, tg):
    t_m, m = [], []
    for i in range(num):
        m.append(Element(pairing, GT))
        t_m.append(Element.random(pairing, G2))
        
    m = [pairing.apply(tg, t_m[i]) for i in range(num)]
    
    return m

m_a = init(10, g)
m_a

[(0x0435ADCD902F7FC59541A433944ED654CDC50763247F54800340C38117F760A180F2417E3DB455F149E19CA7904676AD579BD8E9CF36FC891FE287EF473A3E35, 0x4371341891A6864B0DD09D9C6A08DEF437B99C1CB4C17CEE01A2BB70EC500F85EEBEB94097CCB09DF013A1DBC9B0C72B70178AA3E0D91EFDA7F2651C6F183317),
 (0x680D9A6A6C2742FD3200F6F172C3EA5A535017726C7AB3A91247E5508324D8289C3113E08BCEB7F1DB4F1053C3C055306008FC7D03EAAA68613FFE27D07E47B5, 0x7C278B5110A8A189D2AF764AE007D16E189ADAF2B472EB091EC2322668F8BD0127A9DAE62348D796F877057E6DCA76D568002EF19E3971247AAFA1DAB6C83944),
 (0x5C5C7163023A0A5BD507BB42EC9E9044AF7D4A0F215F24F52DB357EB1928FC4C8F85BAB1DE9DC4C372C770A3B01423D6E90A1F2E53B1B98F2A3F3027AE60F3E4, 0x1B8D3BD147FB2D4EAF4EDBCE168E22496FB8495210607A0CADE34CDC1C3A9FB9183F61CC2175FE81D6D623614C48C88E204A0894EE4931A313280F1FF3C269C0),
 (0x6963105F4E5F336172817AB3EF16FB6922036725229682BA11BD4542E7AF935B48B9007D35DC5E201D7D2F09314F5DF4AA8DCA49F6704F7F76E1C3E76C5680E5, 0x1EE29710AD0D9418EB25CFB3308D24BEE772D8239EC41AD0F335EE1E2229576

In [5]:
z_time = 0.0
p_time = 0.0
c = Element(pairing, GT)

for i in range(100):
    a = Element.random(pairing, G1)
    b = Element.random(pairing, G1)
    r = Element.random(pairing, Zr)
    
    st = time.time()
    d = m_a[0]**r
    z_time += time.time() - st
    
    start = time.time()
    c = pairing.apply(a, b)
    p_time += time.time() - start

print('Z Time: ', z_time / 100)
print('P Time: ', p_time / 100)

Z Time:  0.0005621600151062011
P Time:  0.004771120548248291


In [6]:
def dkg(t_zeta):
    dpk = Element(pairing, G2)
    
    alpha = Element.random(pairing, Zr)
    dpk = t_zeta**alpha
    
    return dpk, alpha

dpk, alpha = dkg(zeta)
alpha

0x52C03D7AEED85A1D254A208798D7FA8F1F3B5AEF

In [7]:
def ukg(tg, th):
    sk, pk = [], []
    for i in range(6):
        sk.append(Element.random(pairing, Zr))

    pk = [(tg**sk[i]) * (th**sk[i+1]) for i in range(0, 6, 2)]
    
    return sk, pk

sk, pk = ukg(g, h)
sk

[0x448D2ED9ACC47C17ABFE2C2A68E0B346A1677B4B,
 0x2628CE4CA2D83C85E3CF00B2ED3864A06FBD6212,
 0x75E819852A3D1F08C6854688880939E91B6503DA,
 0x295C6A2F74DE27DD6137B19E2575338610A16CB3,
 0x473B61B212030F23741701581B4933BD18F099C3,
 0x18E6578D477D7EDB114C2427DF9295ACE345D6D2]

In [9]:
def tkg(tdpk, tsk):
    tk = []
    for i in range(6):
        tk.append(Element(pairing, G2))
    
    tk = [tdpk**tsk[i] for i in range(6)]
    
    return tk

tk = tkg(dpk, sk)
tk

[036FA3D8584D641E43863199314F2AF5B61D44A9D69C80EE99A648834F4AB290FE582A949DBA3DAA3D9A69313112834B018246584F86E3019A37F69B74485824F2,
 02339EC94D9369DA844401093EA10B186710568A37E3A306C3C19C85B8EC9DD83513A23442A40128DEDD6043CF7F0C528CAA5712D5A08FADB85550484ADB39C700,
 034C4C097804C23F8AA3BE5078481F1CD916EACE354D44F44A89B8827AD73F60708C642B4A39B8BE80786E5106A6C74A79438687ACD2A3D66893625ECCE2A6D6C8,
 03106E45D4A867C014A1EDFEF4E22A3888179699FFF38897952F7CD3D57C0BFE40DEDA12E3AC402F2C321F8E5F4B955193365FB88FE11BB62737F51FF0CF2D86AC,
 0292FF1DF37864FCD7170B8EECCDDE407BD96217E7D48153E38CBF22658C3225E4BCC998E7E8DB020FFBA03AABDD95D578B88D1E4DFA45731848C6A6C1C7E29E60,
 034CCBD3EC1EE51D788D65FC3BDF2079CB3BFA6760D4424B02FB3E678F231607E1AA4BC38711DA1D014F438AEF8EDB09FC76C4484F6236CD74586111F37E2543F0]

In [10]:
sk1, pk1 = ukg(g, h)
tk1 = tkg(dpk, sk1)

sk2, pk2 = ukg(g, h)
tk2 = tkg(dpk, sk2)
tk1

[03A3B33A435423C733944994720AA4180A724A92B7A1C13EBF45AA0CFD236D564352A351409CFB678400A79F065EEC82D605CE739212095F78A901BFFD72E08235,
 026A69F259F66B2791CEC5A98916D5C958F8896C0D33A820AF07639F7E70D75A9A77EC333212451AFBF37AC07B9442EE112D825395C5AB5F9E967090643026C941,
 031476A949A26E494B548BE7687FFEF2599FB8022699A68F8AFFFDE0F460194428A3DA6D07DB3759DAAF5DB1EC9D2EB55F6E1012ABE2782EE561BED7340178AFE6,
 0373A1DF5C14B87A44812AA9451F7BF901D9C41D99A3818716C5F1A5CD453C9E4C5F2173877E10B9DE3A0E30E4D4D61E0A68FEBC52F86A570426DED820753364F3,
 025DA8F27A066E08B6899338A8BB6E405939C38FB54D09A6C69194B4CE0A98088E25704DE98654F1C29094F872E99EE78CA707AFFD7CB397486639635B5A093DF9,
 0213A9A543E76D13E174D23EA72015E5AA87C1131A0527131F409C1BF6E3A5544EEB9946EAE22A85162D0ABDBD671E2ED9A7D9FEA9122BF70B0D6967453849E381]

In [11]:
def get_hash(c1, c2, c3):
    h_in = str(c1) + str(c2) + str(c3)
    sha512 = hashlib.sha512()
    sha512.update(h_in.encode('utf8'))
    hv = sha512.hexdigest()
    ev = Element.from_hash(pairing, Zr, hv)
    
    return ev

r = Element.random(pairing, Zr)
        
w1 = g**r
w2 = h**r

x = Element(pairing, GT)
x = pairing.apply(pk1[0], zeta)

theta = get_hash(w1, w2, x)
theta

0x3061633231666537313933346437653466653036

In [12]:
def enc(num, tg, th, t_zeta, tpk, ttk, tsk, tm):
    one = Element.one(pairing, Zr)
    start = time.time()
    c = []
    for i in range(num):
        r = Element.random(pairing, Zr)
        
        w1 = tg**r
        w2 = th**r

        e = pairing.apply(tpk[0]**r, zeta)
        x = (pairing.apply(tpk[0]**r, zeta)) * tm[i] # e((g^s * h^t)^r, zeta)
        
        print((e**(-one)*x) == tm[i])
        
        e1 = pairing.apply(w1, ttk[0])
        e2 = pairing.apply(w2, ttk[1])
        
        theta = get_hash(w1, w2, x)
        y = pairing.apply((tpk[1]**r) * (tpk[2]**theta)**r, zeta)
        
        e3 = pairing.apply(w1, ttk[2])
        e4 = pairing.apply(w2, ttk[3])
        e5 = pairing.apply(w1, ttk[4])
        e6 = pairing.apply(w2, ttk[5])
        e7 = e3 * e4 * (e5**theta) * (e6**theta)
#         print((y**alpha) == e7)
        
        c.append([w1, w2, x, y])
    
    print("Enc Time: ", time.time() - start)
    return c

c1 = enc(10, g, h, zeta, pk1, tk1, sk1, m_a)
c2 = enc(10, g, h, zeta, pk2, tk2, sk2, m_a)


True
True
True
True
True
True
True
True
True
True
Enc Time:  0.8839797973632812
True
True
True
True
True
True
True
True
True
True
Enc Time:  1.1457939147949219


In [13]:
one = Element.one(pairing, Zr)
print((c1[0][2] * (c2[0][2]**-one))**alpha)
print((c1[0][3] * (c2[0][3]**-one)))

(0x88617C42CD7B304AB5B09BD3E48211F46F65523A09118DF174DC921BFC0E53447752A516EA5684F7F144FBC8E20F03E2C996C61545FCDDF04EF7D57C6A376B41, 0x51CC434CFC2A9E6C8C21E9C2C6889082FD18CF025D6517BB42096E8F1BC33B095466F949877058D562E1A1F8AD908050C360EDEC1703E939464F3B0B7E18F1A9)
(0x397788842C5AF03D45A815D3266361CC3B6F3852A1091E5BE61AF2F50B38794F5B5D5C91D8FED10455E0AA15F43C71B68319D715364DE0EE9C935E1E2D4F1456, 0x1DCA806C325E6CCEEE57AC97418A38FA5396941C2231DF4BC44CAD891A2AB831740EC8CA82AF3E4AA68372C1EFA882F12DDC165FA8AD0AB9DE2C67E1EE995991)


In [14]:
def dec(tsk, tc):
    m = []
    one = Element.one(pairing, Zr)
    
    for i in range(len(tc)):
        w1 = tc[i][0]
        w2 = tc[i][1]
        x = tc[i][2]
        y = tc[i][3]
        
        theta = get_hash(w1, w2, x)
        ty1 = (w1**(tsk[2] + theta*tsk[4]))
        ty2 = (w2**(tsk[3] + theta*tsk[5]))
        e1 = pairing.apply((ty1 * ty2), zeta)
        
        if y != e1:
            return None
        else:
            tx = (w1**tsk[0]) * (w2**tsk[1])
            e2 = pairing.apply(tx, zeta)
            m.append(x * (e2**(-one)))
#             print(m)
    
    return m

m_b = dec(sk1, c1)
for i in range(len(m_a)):
    print(m_a[i] == m_b[i])

True
True
True
True
True
True
True
True
True
True


In [15]:
def IVgen(tc1, tc2, ttk1, ttk2):
    v = []
    count = 0
    one = Element.one(pairing, Zr)
    
    for i in range(len(c1)):
        for j in range(len(c2)):
            print(i, j)
            
            omega1 = Element.random(pairing, Zr)
            omega2 = Element.random(pairing, Zr)
        
            w1_1 = tc1[i][0]
            w2_1 = tc1[i][1]
            x_1 = tc1[i][2]
            y_1 = tc1[i][3]
        
            w1_2 = tc2[j][0]
            w2_2 = tc2[j][1]
            x_2 = tc2[j][2]
            y_2 = tc2[j][3]
    
            v1 = (x_1 * (x_2**(-one)))**omega1
        
            e1 = pairing.apply(w1_1, ttk1[0])
            e2 = pairing.apply(w2_1, ttk1[1])
            e3 = pairing.apply(w1_2, ttk2[0])
            e4 = pairing.apply(w2_2, ttk2[1])
        
            v2 = ((e1 * e2) * ((e3 * e4)**(-one)))**omega1
        
            print("x: ", (v1**alpha) == v2)
            if (v1**alpha) == v2:
                count += 1
        
            v3 = (y_1 * (y_2**(-one)))**omega2
        
            theta1 = get_hash(w1_1, w2_1, x_1)
            theta2 = get_hash(w1_2, w2_2, x_2)
        
            e5 = pairing.apply(w1_1, ttk1[2])
            e6 = pairing.apply(w2_1, ttk1[3])
            e7 = (pairing.apply(w1_1, ttk1[4]))
            e8 = (pairing.apply(w2_1, ttk1[5]))
            temp_y1 = e5 * e6 * (e7**theta1) * (e8**theta1)
        
            e9 = pairing.apply(w1_2, ttk2[2])
            e10 = pairing.apply(w2_2, ttk2[3])
            e11 = (pairing.apply(w1_2, ttk2[4]))
            e12 = (pairing.apply(w2_2, ttk2[5]))
            temp_y2 = e9 * e10 * (e11**theta2) * (e12**theta2)
        
            print("y: ", (y_2**alpha) == temp_y2)
        
            v4 = (temp_y1 * (temp_y2**(-one)))**omega2
            v.append([v1, v2, v3, v4])
    print("count: ", count)
    return v

v = IVgen(c1, c2, tk1, tk2)
print(len(v))

0 0
x:  True
y:  True
0 1
x:  False
y:  True
0 2
x:  False
y:  True
0 3
x:  False
y:  True
0 4
x:  False
y:  True
0 5
x:  False
y:  True
0 6
x:  False
y:  True
0 7
x:  False
y:  True
0 8
x:  False
y:  True
0 9
x:  False
y:  True
1 0
x:  False
y:  True
1 1
x:  True
y:  True
1 2
x:  False
y:  True
1 3
x:  False
y:  True
1 4
x:  False
y:  True
1 5
x:  False
y:  True
1 6
x:  False
y:  True
1 7
x:  False
y:  True
1 8
x:  False
y:  True
1 9
x:  False
y:  True
2 0
x:  False
y:  True
2 1
x:  False
y:  True
2 2
x:  True
y:  True
2 3
x:  False
y:  True
2 4
x:  False
y:  True
2 5
x:  False
y:  True
2 6
x:  False
y:  True
2 7
x:  False
y:  True
2 8
x:  False
y:  True
2 9
x:  False
y:  True
3 0
x:  False
y:  True
3 1
x:  False
y:  True
3 2
x:  False
y:  True
3 3
x:  True
y:  True
3 4
x:  False
y:  True
3 5
x:  False
y:  True
3 6
x:  False
y:  True
3 7
x:  False
y:  True
3 8
x:  False
y:  True
3 9
x:  False
y:  True
4 0
x:  False
y:  True
4 1
x:  False
y:  True
4 2
x:  False
y:  True
4 3
x:  False
y

In [16]:
def pTest(num, tc1, tc2, tpk1, tpk2, ttk1, ttk2, tdpk):
    v = []
    start = time.time()

    for i in range(num):
        e1 = pairing.apply(tpk1[0], tdpk)
        e2 = pairing.apply(g, ttk1[0])
        e3 = pairing.apply(h, ttk1[1])
        b1 = (e1 == (e2 * e3))
        
        e4 = pairing.apply(tpk1[1], tdpk)
        e5 = pairing.apply(g, ttk1[2])
        e6 = pairing.apply(h, ttk1[3])
        b2 = (e4 == (e5 * e6))
        
        e7 = pairing.apply(tpk1[2], tdpk)
        e8 = pairing.apply(g, ttk1[4])
        e9 = pairing.apply(h, ttk1[5])
        b3 = (e7 == (e8 * e9))
        
        e10 = pairing.apply(tpk2[0], tdpk)
        e11 = pairing.apply(g, ttk2[0])
        e12 = pairing.apply(h, ttk2[1])
        b4 = (e10 == (e11 * e12))
        
        e13 = pairing.apply(tpk2[1], tdpk)
        e14 = pairing.apply(g, ttk2[2])
        e15 = pairing.apply(h, ttk2[3])
        b5 = (e13 == (e14 * e15))
        
        e16 = pairing.apply(tpk2[2], tdpk)
        e17 = pairing.apply(g, ttk2[4])
        e18 = pairing.apply(h, ttk2[5])
        b6 = (e16 == (e17 * e18))
        
        b7 = (b1 and b2 and b3 and b4 and b5 and b6)
        if b7:
            IVgen(tc1, tc2, ttk1, ttk2)
    
pTest(10, c1, c2, pk1, pk2, tk1, tk2, dpk)

0 0
x:  True
y:  True
0 1
x:  False
y:  True
0 2
x:  False
y:  True
0 3
x:  False
y:  True
0 4
x:  False
y:  True
0 5
x:  False
y:  True
0 6
x:  False
y:  True
0 7
x:  False
y:  True
0 8
x:  False
y:  True
0 9
x:  False
y:  True
1 0
x:  False
y:  True
1 1
x:  True
y:  True
1 2
x:  False
y:  True
1 3
x:  False
y:  True
1 4
x:  False
y:  True
1 5
x:  False
y:  True
1 6
x:  False
y:  True
1 7
x:  False
y:  True
1 8
x:  False
y:  True
1 9
x:  False
y:  True
2 0
x:  False
y:  True
2 1
x:  False
y:  True
2 2
x:  True
y:  True
2 3
x:  False
y:  True
2 4
x:  False
y:  True
2 5
x:  False
y:  True
2 6
x:  False
y:  True
2 7
x:  False
y:  True
2 8
x:  False
y:  True
2 9
x:  False
y:  True
3 0
x:  False
y:  True
3 1
x:  False
y:  True
3 2
x:  False
y:  True
3 3
x:  True
y:  True
3 4
x:  False
y:  True
3 5
x:  False
y:  True
3 6
x:  False
y:  True
3 7
x:  False
y:  True
3 8
x:  False
y:  True
3 9
x:  False
y:  True
4 0
x:  False
y:  True
4 1
x:  False
y:  True
4 2
x:  False
y:  True
4 3
x:  False
y

In [17]:
def dTest(tv, t_alpha):
    if((tv[2]**t_alpha) != tv[3]):
        return None
    else:
        if((tv[0]**t_alpha) != tv[1]):
            return 0
        else:
            return 1

# print(v)
for i in range(len(v)):
    print(dTest(v[i], alpha))

1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1


In [19]:
json_file = '/home/seed/Documents/ResearchPaper/discriminator.json'
with open(json_file, 'r') as f:
    d_info = json.load(f)

dsk_s = d_info['dsk']
dsk_v = int(dsk_s, 16)
dsk = Element(pairing, Zr, value=dsk_v)
dsk

NameError: name 'json' is not defined